In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, LSTM, Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
import sys, os, time, json, datetime, glob

from logzero import logger
import warnings
#import tensorflow as tf

import numpy as np
import pandas as pd
from pathlib import Path

from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

from art.metrics import RobustnessVerificationTreeModelsCliqueMethod

from art.attacks.evasion import FastGradientMethod, SaliencyMapMethod
from art.estimators.classification import KerasClassifier

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/estimators/certification/__init__.py:28: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [2]:
model = Sequential()
model.add(Dense(64, input_dim = 19, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(41, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [3]:
df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/features_nov-apr.csv", parse_dates = ['time_start'])
df.fillna(0, inplace=True)



/var/tmp/pbs.8254068.pbs/ipykernel_3720150/4075129579.py:1: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/features_nov-apr.csv", parse_dates = ['time_start'])


In [4]:
y = []
packets = [] 
i = 0
    
devs = ['appkettle','blink-security-hub','bosiwo-camera-wifi','lefun-cam-wired','insteon-hub',
            'echoplus','meross-dooropener','smartlife-bulb','xiaomi-ricecooker','ubell-doorbell',
            'appletv','tplink-bulb','google-home','icsee-doorbell','t-wemo-plug','echospot',
            'nest-tstat','sousvide','smartlife-remote','netatmo-weather-station','lgtv-wifi',
            'wansview-cam-wired','xiaomi-plug','xiaomi-hub','lightify-hub','bosiwo-camera-wired',
            'tplink-plug2','allure-speaker','honeywell-thermostat','smarter-coffee-mach','roku-tv',
            'yi-camera','firetv','echodot','smartthings-hub','reolink-cam-wired','t-philips-hub',
            'switchbot-hub','ring-doorbell','blink-camera','samsungtv-wired']

features = ['srcPort', 'destPort',
       'bytes_out', 'num_pkts_out', 'bytes_in', 'num_pkts_in', 'f_ipt_mean',
       'f_ipt_std', 'f_ipt_var', 'f_ipt_skew', 'f_ipt_kurtosis', 'f_b_mean',
       'f_b_std', 'f_b_var', 'f_b_skew', 'f_b_kurtosis', 'duration', 'pr',
       'domainId']
label = 'deviceId'

#weeksToProcess = list(range(44,53)) + list(range(1,19))
weeks = [list(range(44,53)), list(range(1,10)), list(range(10,19))]
weeksToProcess = weeks[0] + weeks[1] + weeks[2]
#weeksToProcess = weeks[0]

In [5]:
trainCon = df['time_start'].dt.isocalendar().week.isin(weeks[0])


In [6]:
testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1Y').tolist()
lb = LabelBinarizer()
lb.fit(range(1,41))

LabelBinarizer()

In [7]:
model = keras.models.load_model("model.15-0.43.h5", compile=False)
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
classifier = KerasClassifier(model=model, clip_values=(0, 1))

2023-09-19 10:55:48.710068: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-09-19 10:55:48.710520: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 10:55:48.731164: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 32. Tune using inter_op_parallelism_threads for best performance.
2023-09-19 10:55:48.740536: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2023-09-19 10:55:48.753918: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2249970000 Hz


In [12]:

attack_fgsm = SaliencyMapMethod(classifier=classifier, batch_size=1)
for date in testCons:
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    #testDF = df[testCon]
    testDF = df

    X = testDF[features]
    y = testDF[label]

    if len(X) == 0:
        continue
    
    print("starting scalar")

    scaler = StandardScaler().fit(df[trainCon][features].values)
    X = scaler.transform(X.values)
    
    
            
    labels = lb.transform(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=4800, stratify=y)
 
    print("labelled scalar") 
    labels = lb.transform(y_test)
    
    x_test_adv = attack_fgsm.generate(X_test)
    np.savetxt(f"nnjsma-{date.week}.csv", x_test_adv, delimiter=",")
    print("fgsm generated")
    loss_test, accuracy_test = model.evaluate(x_test_adv, y_test-1)
    perturbation = np.mean(np.abs((x_test_adv - X_test)))
    print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
    print('Average perturbation: {:4.2f}'.format(perturbation))
    for index, sample in enumerate (x_test_adv):
        dist = dist + np.linalg.norm(sample-X_test[index])
    print(dist/4800)

starting scalar
labelled scalar


JSMA: 100%|██████████| 4800/4800 [06:20<00:00, 12.61it/s]


fgsm generated
Accuracy on adversarial test data: 7.90%
Average perturbation: 0.78
7.910303745070824


In [13]:
dist=0
for index, sample in enumerate (x_test_adv):
    dist = dist + np.linalg.norm(sample-X_test[index])
print(dist/4800)

4.059913425003889


In [31]:
#y_test = np_utils.to_categorical(y_test, 41)
y_test = y_test-1
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - X_test)))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))


Accuracy on adversarial test data: 7.12%


In [36]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
X_test = normalize(X=X_test)

rt = RobustnessVerificationTreeModelsCliqueMethod(classifier=classifier)
average_bound, verified_error = rt.verify(x=X_test, y=y_test, eps_init=0.3, nb_search_steps=10, max_clique=2, 
                                          max_level=2)

print('Average bound:', average_bound)
print('Verified error at eps:', verified_error)

AttributeError: 'KerasClassifier' object has no attribute 'get_trees'

In [37]:
num_samples=100
indexes = np.array(range(num_samples))
extra_model = RandomForestClassifier()

extra_classifier = ScikitlearnRandomForestClassifier(extra_model)
leakage, _, _ = PDTP(art_classifier, extra_classifier, x_train, y_train, indexes=indexes)

print("Average PDTP leakage random forest: ", np.average(leakage))
print("Max PDTP leakage random forest: ", np.max(leakage))

NameError: name 'RandomForestClassifier' is not defined